In [2]:
import pandas as pd

In [8]:
skaters_df=pd.read_csv('skaters_2009_2023.csv')
skaters_df.reset_index(drop=True, inplace=True)
skaters_df.columns

Index(['Unnamed: 0', 'Rk', 'Player', 'Age', 'Tm', 'Pos', 'GP', 'G', 'A', 'PTS',
       '+/-', 'PIM', 'PS', 'EV', 'PP', 'SH', 'GW', 'EV.1', 'PP.1', 'SH.1', 'S',
       'S%', 'TOI', 'ATOI', 'BLK', 'HIT', 'FOW', 'FOL', 'FO%', 'SEASON'],
      dtype='object')

In [47]:
def positional_definition(value):
    if value=="C":
        return 1
    elif value=="RW":
        return 2
    elif value=="LW":
        return 3
    elif value=="F" or value=="W":
        return 2
    elif value=="D":
        return 4

In [159]:
key_features=['Player', 'Age', 'Pos', 'GP', 'G', 'A', 'PTS', '+/-','PIM','EV','PP','SH','BLK','HIT','SEASON', 'PS']
number_features=['Age', 'Pos', 'GP', 'G', 'A', 'PTS', '+/-','PIM','EV','PP','SH','BLK','HIT','SEASON', 'PS']
normalized_features=['Player','Age', 'Pos', 'GP','G/GP','A/GP','PTS/GP', 'PM/GP','PIM/GP', 'EV/GP', 'PP/GP','SH/GP','BLK/GP','HIT/GP', 'SEASON','PS']
smaller_df=skaters_df[key_features]
smaller_df=smaller_df.dropna()
smaller_df=smaller_df.loc[smaller_df['Age']!='Age']
smaller_df['Pos']=smaller_df['Pos'].apply(positional_definition)
smaller_df[number_features]=smaller_df[number_features].astype(float)
smaller_df['G/GP']=smaller_df['G']/smaller_df['GP']
smaller_df['A/GP']=smaller_df['A']/smaller_df['GP']
smaller_df['PTS/GP']=smaller_df['PTS']/smaller_df['GP']
smaller_df['PM/GP']=smaller_df['+/-']/smaller_df['GP']
smaller_df['PIM/GP']=smaller_df['PIM']/smaller_df['GP']
smaller_df['EV/GP']=smaller_df['EV']/smaller_df['GP']
smaller_df['PP/GP']=smaller_df['PP']/smaller_df['GP']
smaller_df['SH/GP']=smaller_df['SH']/smaller_df['GP']
smaller_df['BLK/GP']=smaller_df['BLK']/smaller_df['GP']
smaller_df['HIT/GP']=smaller_df['HIT']/smaller_df['GP']
normalized_df=smaller_df[normalized_features]
normalized_df

,Player,Age,Pos,GP,G/GP,A/GP,PTS/GP,PM/GP,PIM/GP,EV/GP,PP/GP,SH/GP,BLK/GP,HIT/GP,SEASON,PS
0,Nicholas Abruzzese,23.0,1.0,2.0,0.000000,1.000000,1.000000,0.500000,0.000000,0.000000,0.000000,0.00,0.000000,0.500000,2023.0,0.2
1,Noel Acciari,31.0,1.0,77.0,0.181818,0.116883,0.298701,-0.051948,0.272727,0.181818,0.000000,0.00,1.051948,3.168831,2023.0,1.5
2,Calen Addison,22.0,4.0,62.0,0.048387,0.419355,0.467742,-0.274194,0.354839,0.048387,0.000000,0.00,0.612903,0.306452,2023.0,3.2
3,Egor Afanasyev,22.0,3.0,17.0,0.058824,0.000000,0.058824,-0.411765,0.117647,0.058824,0.000000,0.00,0.764706,1.352941,2023.0,-0.3
4,Andrew Agozzino,32.0,3.0,4.0,0.250000,0.500000,0.750000,-0.250000,0.000000,0.000000,0.000000,0.25,1.000000,3.000000,2023.0,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13499,Marek Židlický,31.0,4.0,76.0,0.157895,0.394737,0.552632,-0.157895,1.000000,0.026316,0.131579,0.00,1.250000,0.973684,2009.0,7.8
13500,Mike Zigomanis,28.0,1.0,22.0,0.090909,0.181818,0.272727,-0.090909,1.227273,0.090909,0.000000,0.00,0.363636,0.772727,2009.0,0.3
13501,Ilya Zubov,21.0,1.0,10.0,0.000000,0.200000,0.200000,-0.100000,0.000000,0.000000,0.000000,0.00,0.200000,1.100000,2009.0,0.0
13502,Sergei Zubov*,38.0,4.0,10.0,0.000000,0.400000,0.400000,-0.400000,0.000000,0.000000,0.000000,0.00,0.300000,0.600000,2009.0,0.3


In [160]:
def find_nearest_neighbours(df, player, age):
    relevant_stats=['Age', 'Pos','GP', 'G/GP','A/GP','PTS/GP','PM/GP','PIM/GP','EV/GP','PP/GP','SH/GP','BLK/GP','HIT/GP']
    player_stats=df.loc[(df['Age']==age) & (df['Player']==player)]
    season=float(player_stats['SEASON'])
    player_stats=player_stats[relevant_stats]
    similar_ages=df.loc[(df['Age']==age) & (df['Player']!=player) & (df['SEASON']<=season)]
    similar_names=list(similar_ages['Player'])
    point_shares=list(similar_ages['PS'])
    similar_ages=similar_ages[relevant_stats]
    # return similar_ages.to_numpy()
    differences=similar_ages.to_numpy()-player_stats.to_numpy()
    differences=pd.DataFrame(differences, columns=relevant_stats)
    differences['similarity_score']=100-(abs(differences['Pos'])*40+abs(differences['GP'])*5+abs(differences['G/GP'])*30+abs(differences['A/GP'])*20+abs(differences['PTS/GP'])*30+abs(differences['PM/GP'])*5+abs(differences['PIM/GP'])*0.25+abs(differences['EV/GP'])*40+abs(differences['PP/GP'])*40+abs(differences['SH/GP'])*10+abs(differences['BLK/GP'])*0.1+abs(differences['HIT/GP'])*2)
    differences['Player']=similar_names
    differences['PS']=point_shares
    return differences.loc[differences['similarity_score']>0].sort_values(by='similarity_score', ascending=False), season

In [190]:
def determine_player_projections(df, player, age, number_of_years_from_now):
    stored, season=find_nearest_neighbours(df, player, age)
    list_names=list(stored['Player'])
    stored_sim_scores=stored[['Player','similarity_score']]
    projecting_df=df.loc[(df['Player'].isin(list_names)) & (df['Age']==age+number_of_years_from_now) & (df['SEASON']<=season)]
    projecting_df=projecting_df.merge(stored_sim_scores, how='left', on='Player')
    similarity_score_sum=projecting_df['similarity_score'].sum()
    total=(projecting_df['similarity_score']*projecting_df['PS']).sum()
    projected_ps=total/similarity_score_sum
    return projected_ps

In [186]:
stored=find_nearest_neighbours(normalized_df, 'Auston Matthews', 25)
stored

/var/folders/pq/qr32tkk90n7361gslj40w8j4tyz0cb/T/ipykernel_15765/178309995.py:4: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  season=float(player_stats['SEASON'])


(      Age  Pos   GP      G/GP      A/GP    PTS/GP     PM/GP    PIM/GP  \
 1070  0.0  0.0  0.0 -0.094595 -0.229730 -0.324324 -0.391892  0.243243   
 943   0.0  0.0  1.0  0.126126  0.178559  0.304685 -0.178919  0.663063   
 69    0.0  0.0  4.0  0.062024 -0.005544  0.056480 -0.367637  0.229730   
 387   0.0  0.0  3.0 -0.085995 -0.101615 -0.187610 -0.353984 -0.036504   
 131   0.0  0.0 -3.0 -0.103921 -0.072897 -0.176818 -0.672440  0.391702   
 ...   ...  ...  ...       ...       ...       ...       ...       ...   
 288   0.0  0.0 -6.0 -0.467011 -0.358108 -0.825119 -0.404213 -0.064388   
 62    0.0  0.0 -9.0 -0.325156 -0.361954 -0.687110 -0.265073  0.098960   
 604   0.0  1.0  8.0 -0.064931 -0.169084 -0.234015 -0.431114 -0.123929   
 391   0.0  0.0  8.0 -0.357614 -0.388596 -0.746210 -0.626236 -0.001978   
 1016  0.0  0.0  4.0 -0.463617 -0.531185 -0.994802 -0.598406  0.755371   
 
          EV/GP     PP/GP     SH/GP    BLK/GP    HIT/GP  similarity_score  \
 1070 -0.094595 -0.027027  0.0270

In [204]:
determine_player_projections(normalized_df, 'Daniel Sprong', 25, 1)

/var/folders/pq/qr32tkk90n7361gslj40w8j4tyz0cb/T/ipykernel_15765/178309995.py:4: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  season=float(player_stats['SEASON'])


3.576485339831804

In [ ]:
team_colorado=['Artturi Lehkonen', 'Nathan Mackinnon', 'Valeri Nichushkin', 'Tomas Tatar', 'Ryan Johansen', 'Mikko Rantanen', 'Jonathan Drouin', 'Ross Colton', 'Miles Wood', 'Andrew Cogliano', 'Frederik Olofsson', 'Logan O\'Connor', 'Devon Toews', 'Cale Makar', 'Bowen Byram', 'Samuel Girard', 'Jack Johnson', 'Josh Manson', 'Alexander Georgiev', 'Pavel Francouz']
team_dallas=['Jason Robertson', 'Roope Hintz', 'Joe Pavelski', 'Jamie Benn', 'Wyatt Johnston', 'Yevgeni Dadonov', 'Mason Marchment', 'Matt Duchene', 'Tyler Seguin', 'Ty Dellandrea', 'Radek Faksa', 'Craig Smith', 'Miro Heiskanen', 'Ryan Suter', 'Thomas Harley', 'Jani Hakanpaa', 'Nils Lundkvist', 'Esa Lindell', 'Jake Oettinger', 'Scott Wedgewood']
team_coyotes=['Barrett Hayton', 'Nick Schmaltz', 'Clayton Keller', 'Matias Maccelli', 'Logan Cooley', 'Jan Jenik','Lawson Crouse', 'Nick Bjustad', 'Zach Sanford', 'Jason Zucker', 'Alexander Kerfoot', 'Jack McBain', 'Janis Moser', 'Sean Durzi', 'Matt Dumba', 'Travis Dermott', 'Juuso Valimaki','Troy Stetcher', 'Connor Ingram', 'Karel Vejmalka']
team_predators=['Filip Forsberg', 'Ryan O\'Reilly', 'Juuso Parssinen', 'Gustav Nyquist', 'Colton Sissons', 'Samuel Fagemo', 'Luke Evangelista', 'Tommy Novak', 'Kiefer Sherwood', 'Yakov Trenin', 'Michael Mccarron', 'Cole Smith', 'Dante Fabbro', 'Roman Josi', 'Alexandre Carrier', 'Ryan Mcdonagh', 'Tyson Barrie', 'Jeremy Lauzon', 'Juuse Saros', 'Kevin Lankinen']
team_blues=['Brandon Saad', 'Robert Thomas', 'Jordan Kyrou', 'Pavel Buchnevich', 'Brayden Schenn', 'Kasperi Kapanen', 'Jakub Vrana', 'Kevin Hayes', 'Samuel Blais', ' Jake Neighbours', 'Oskar Sundkvist', 'Alexei Toropchenko', 'Nick Leddy', 'Colton Parayko', 'Torey Krug', 'Justin Faulk', 'Tyler Tucker', 'Marco Scandella', 'Jordan Binnington', 'Joel Hofer']
team_wild=['Kirill Kaprizov', 'Ryan Hartman', 'Mats Zuccarello', 'Pat Maroon', 'Joel Eriksson Ek', 'Marcus Johansson', 'Marcus Foligno', 'Marco Rossi', 'Frederick Gaudreau', 'Matt Boldy', 'Connor Dewar', 'Brandon Duhaime', 'Jonas Brodin', 'Calen Addison', 'Jacob Middleton', 'Brock Faber', 'Jared Spurgeon', 'Jon Merrill', 'Filip Gustavsson', 'Marc-Andre Fleury']
team_jets=['Kyle Connor', 'Mark Scheiffle', 'Alex Iafallo', 'Nikolaj Ehlers', 'Vladislav Namestnikov', 'Cole Perfetti', 'Nino Niederreiter', 'Adam Lowry', 'Mason Appleton', 'David Gustafsson', 'Rasmus Kupari', 'Morgan Barron', 'Dylan Demelo', 'Josh Morrissey', 'Brendan Dillon', 'Neal Pionk', 'Nate Schmidt', 'Dylan Samberg', 'Connor Hellebuyck', 'Laurent Brossoit']
team_blackhawks=['Connor Bedard', 'Tyler Johnson', 'Nick Foligno', 'Andreas Athanasiou', 'Lukas Reichel', 'Taylor Raddysh', 'Corey Perry', 'Jason Dickinson', 'Philipp Kurashev', 'Boris Katchouk', 'Ryan Donato', 'Reese Johnson', 'Kevin Korchinski', 'Seth Jones', 'Alex Vlasic', 'Connor Murphy', 'Wyatt Kaiser', 'Jarred Tinordi', 'Petr Mrazek', 'Arvid Soderblom']
team_knights=['Ivan Barbashev', 'Jack Eichel', 'Jonathan Marchessault', 'Mark Stone', 'Chandler Stephenson', 'Paul Cotter', 'Brett Howden', 'William Karlsson', 'Michael Amadio', 'Keegan Kolesar', 'Nicolas Roy', 'William Carrier', 'Nicolas Hague', 'Alec Martinez', 'Shea Theodore', 'Brayden McNabb', 'Alex Pietrangelo', 'Brayden Pachal', 'Logan Thompson', 'Adin Hill']
team_canucks=['Phil Di Giuseppe', 'J.T. Miller', 'Brock Boeser', 'Andrey Kuzmenko', 'Elias Pettersson', 'Ilya Mikheyev', 'Dakota Joshua', 'Pius Suter', 'Conor Garland', 'Nils Hoglander', 'Sam Lafferty', 'Anthony Beauvillier', 'Filip Hronek', 'Quinn Hughes', 'Ian Cole', 'Mark Friedman', 'Tyler Myers', 'Carson Soucy', 'Thatcher Demko', 'Casey Desmith']
team_kings=['Anze Kopitar', 'Quinton Byfield', 'Adrian Kempe', 'Kevin Fiala', 'Pierre-Luc Dubois', 'Alex Laferriere', 'Arthur Kailyev', 'Phillip Danault', 'Trevor Moore', 'Carl Grundstrom', 'Blake Lizotte', 'Trevor Lewis', 'Mikey Anderson', 'Drew Doughty', 'Matt Roy', 'Vladislav Gavrikov', 'Jordan Spence', 'Andreas Englund', 'Phoenix Copley', 'Cam Talbot']
team_flames=['Adam Ruzicka', 'Elias Lindholm', 'Matt Coronato', 'Jonathan Huberdeau', 'Nazem Kadri', 'Dillon Dube', 'Andrew Mangiapane', 'Mikael Backlund', 'Blake Coleman', 'Dryden Hunt', 'Yegor Sharangovich', 'Walker Duehr', 'Noah Hanifin', 'Christopher Tanev', 'Nikita Zadorov', 'Mackenzie Weegar', 'Jordan Oesterle', 'Rasmus Andersson', 'Jacob Markstrom', 'Dan Vladar']
team_oilers=['Leon Draisaitl', 'Evander Kane', 'Mattias Janmark', 'Zach Hyman', 'Ryan Nugent-Hopkins', 'Warren Foegle', 'Connor Brown', 'Ryan McLeod', 'Dylan Holloway', 'Adam Erne', 'Derek Ryan', 'Connor McDavid', 'Darnell Nurse', 'Cody Ceci', 'Mattias Ekholm', 'Evan Bouchard', 'Brett Kulak', 'Vincent Desharnais', 'Jack Campbell', 'Stuart Skinner']
team_kraken=['Tye Kartye', 'Matty Beniers', 'Jordan Eberle', 'Oliver Bjorkstrand', 'Alexander Wennberg', 'Jaden Schwartz', 'Eeli Tolvanen', 'Yanni Gourde', 'Jared McCann', 'Kailer Yamamoto', 'Pierre-Edouard Bellemare', 'Andre Burakovsky', 'Vince Dunn', 'Adam Larsson', 'Will Borgen', 'Jamie Oleksiak', 'Justin Schultz', 'Brian Dumoulin', 'Joey Daccord', 'Philipp Grubauer']
team_ducks=['Adam Henrique', 'Trevor Zegras', 'Troy Terry', 'Ryan Strome', 'Mason McTavish', 'Frank Vatrano', 'Max Jones', 'Benoit-Olivier Groulx', 'Jakob Silfverberg', 'Brett Leason', 'Sam Carrick', 'Ross Johnston', 'Cam Fowler', 'Urho Vaakanainen', 'Radko Gudas', 'Ilya Lyubushkin', 'Pavel Mintyukov', 'Jackson Lacombe', 'John Gibson', 'Lukas Dostal']
team_sharks=['Alexander Barabanov', 'Tomas Hertl', 'Anthony Duclair', 'William Eklund', 'Jacob Peterson', 'Kevin Labanc', 'Fabian Zetterlund', 'Nico Sturm', 'Mike Hoffman', 'Filip Zadina', 'Thomas Bordeleau', 'Luke Kunin', 'Jan Rutta', 'Mario Ferraro', 'Matt Benning', 'Kyle Burroughs', 'Nikolai Knyzhov', 'Ty Emberson', 'Mackenzie Blackwood', 'Kaapo Kahkonen']
team_bruins=['Brad Marchand', 'Pavel Zacha', 'David Pastrnak', 'Morgan Geekie', 'Matthew Poitras', 'Jake DeBrusk', 'James Van Riemsdyk', 'Charlie Coyle', 'Trent Frederic', 'Patrick Brown', 'John Beecher', 'Jakub Lauko', 'Matt Grzelcyk', 'Charlie McAvoy', 'Brandon Carlo', 'Hampus Lindholm', 'Derek Forbort', 'Kevin Shattenkirk', 'Jeremy Swayman', 'Linus Ullmark']
team_wings=['Alex DeBrincat', 'Dylan Larkin' , 'Lucas Raymond', 'Andrew Copp', 'J.T. Compher', 'Michael Rasmussen', 'David Perron', 'Joel Veleno', 'Daniel Sprong', 'Christian Fischer', 'Austin Czarnik', 'Jonatan Berggren', 'Jake Walman', 'Moritz Seider', 'Ben Chiarot', 'Justin Holl', 'Shayne Gostisbehere', 'Olli Maatta', 'Ville Husso', 'James Reimer']
team_canadiens=['Cole Caufield', 'Nick Suzuki', 'Rafael Harvey-Pinard', 'Juraj Slafkovsky', 'Alex Newhook', 'Tanner Pearson', 'Josh Anderson', 'Sean Monahan', 'Brendan Gallagher', 'Jesse Ylonen', 'Jake Evans', 'Michael Pezzetta', 'Mike Matheson', 'Johnathan Kovacevic', 'Jordan Harris', 'Kaiden Guhle', 'Justin Barron', 'Arber Xhekaj', 'Cayden Primeau', 'Jake Allen']
team_leafs=['Calle Jarnkrok', 'Auston Matthews', 'Mitchell Marner', 'William Nylander', 'John Tavares', 'Tyler Bertuzzi', 'Matthew Knies', 'David Kampf', 'Max Domi', 'Ryan Reaves', 'Pontus Holmberg', 'Noah Gregor', 'Morgan Reilly', 'T.J. Brodie', 'Timothy Liljegren', 'Jake McCabe', 'John Klingberg', 'Mark Giordano', 'Joseph Woll', 'Ilya Samsonov']
team_senators=['Brady Tkachuk', 'Tim Stutzle', 'Claude Giroux', 'Drake Batherson', 'Josh Norris', 'Dominik Kubalik', 'Mathieu Joseph', 'Ridly Greig', 'Mark Kastelic', 'Rourke Chartier', 'Parker Kelly', 'Thomas Chabot', 'Jake Sanderson', 'Jakob Chychrun', 'Jacob Bernard-Docker', 'Travis Hamonic', 'Erik Brannstrom', 'Anton Forsberg', 'Joonas Korpisalo']
team_lightning=['Alex Barre-Boulet', 'Brayden Point', 'Nikita Kucherov', 'Brandon Hagel', 'Nick Paul', 'Steven Stamkos', 'Tanner Jeannot', 'Anthony Cirelli', 'Waltteri Merela', 'Conor Sheary', 'Luke Glendening', 'Michael Eyssimont', 'Erik Cernak', 'Victor Hedman', 'Michail Sergachev', 'Nick Perbix', 'Darren Raddysh', 'Calvin De Haan', 'Jonas Johansson', ' Andrei Vasilevskiy']
team_panthers=['Evan Rodrigues','Eetu Luostarinen', 'Sam Reinhart', 'Matthew Tkachuk', 'Anton Lundell', 'Carter Verhaeghe', 'Kevin Stenlund', 'Aleksander Barkov', 'Nick Cousins', 'Steven Lorentz', 'Sam Bennett', 'Ryan Lomberg', 'Gustav Forsling', 'Oliver Ekman Larsson', 'Dmitri Kulikov', 'Niko Mikkola', 'Josh Mahura', 'Aaron Ekblad', 'Brandon Montour', 'Sergei Bobrovsky','Anthony Stolarz']
team_sabres=['Tage Thompson', 'Dylan Cozens', 'Jordan Greenway', 'Jeff Skinner', 'Casey Mittelstadt', 'Alex Tuch', 'Victor Olofsson', 'Peyton Krebs', 'JJ Peterka', 'Kyle Okposo', 'Tyson Jost', 'Zemgus Girgensons', 'Mattias Samuelsson', 'Rasmus Dahlin', 'Owen Power', 'Henri Jokiharku', 'Erik Johnson', 'Connor Clifton', 'Ukka-Pekka Luukkonen', 'Eric Comrie']
team_flyers=['Sean Couturier', 'Cam Atkinson', 'Owen Tippett', 'Travis Konecny', 'Scott Laughton', 'Tyson Foerster', 'Bobby Brink', 'Noah Cates', 'Joel Farabee', 'Nicolas Deslauriers', 'Ryan Poehling', 'Garnet Hathaway', 'Cam York', 'Travis Sanheim', 'Yegor Zemula', 'Sean Walker', 'Marc Staal', 'Rasmus Ristolainen', 'Carter Hart', 'Felix Sandstrom']
team_jackets=['Jack Roslovic', 'Adam Fantilli', 'Alexandre Texier', 'Kirill Marchenko', 'Boone Jenner', 'Johnny Gaudreau', 'Kent Johnson', 'Cole Sillinger', 'Emil Bemstrom', 'Patrik Laine', 'Sean Kuraly', 'Justin Danforth', 'Zach Werenski', 'Damon Severson', 'David Jiricek', 'Ivan Provorov', 'Erik Gudbranson', 'Jake Bean', 'Elvis Merzlikins', 'Spencer Martin']
team_rangers=['Chris Kreider', 'Mika Zibanejad', 'Kaapo Kakko', 'Alexis Lafreniere', 'Filip Chytil', 'Artemi Panarin', 'Blake Wheeler', 'Vincent Trochek', 'Will Cuylle', 'Barclay Goodrow', 'Nick Bonino', 'Jimmy Vesey', 'Jacob Trouba', 'K\'Andre Miller', 'Adam Fox', 'Ryan Lindgren', 'Erik Gustafsson', 'Braden Schneider', 'Igor Shestyorkin', 'Jonathan Quick']
team_hurricanes=['Sebastian Aho', 'Teuvo Teravainen', 'Martin Necas', 'Seth Jarvis', 'Jesperi Kotkaniemi', 'Stefan Noesen', 'Jordan Martinook', 'Jordan Staal', 'Jesper Fast', 'Brendan Lemieux', 'Jack Drury', 'Michael Bunting', 'Brent Burns', 'Jaccob Slavin', 'Brady Skjei', 'Jalen Chatfield', 'Tony Deangelo', 'Dmitri Orlov', 'Frederik Andersen', 'Antti Raanta']
team_devils=['Jack Hughes', 'Timo Meier', 'Tyler Toffoli', 'Ondrej Palat', 'Nico Hischier', 'Jesper Bratt', 'Alexander Holtz', 'Erik Haula', 'Dawson Mercer', 'Curtis Lazar', 'Michael McLeod', 'Nathan Bastian', 'Dougie Hamilton', 'Jonas Siegenthaler', 'Kevin Bahl', 'John Marino', 'Luke Hughes', 'Brendan Smith', 'Akira Schmid', 'Vitek Vanecek']
team_islanders=['Anders Lee', 'Bo Horvat', 'Mathew Barzal', 'Kyle Palmieri', 'Brock Nelson', 'Pierre Engvall', 'Simon Holmstrom', 'Jean-Gabriel Pageau', 'Oliver Wahlstrom', 'Cal Clutterbuck', 'Casey Cizikas', 'Matt Martin', 'Noah Dobson', 'Adam Pelech', 'Alexander Romanov', 'Ryan Pulock', 'Sebastian Aho', 'Samuel Bolduc', 'Ilya Sorokin', 'Semyon Varlamov']
team_penguins=['Jake Guentzel', 'Sidney Crosby', 'Bryan Rust', 'Rickard Rakell', 'Yevgeni Malkin', 'Reilly Smith', 'Radim Zohorna', 'Lars Eller', 'Drew O\'Connor', 'Lars Eller', 'Radim Zohorna', 'Jeff Carter', 'Noel Acciari', 'Matt Nieto', 'Kris Letang', 'Ryan Graves', 'Marcus Pettersson', 'Erik Karlsson', 'John Ludvig', 'Ryan Shea', 'Alex Nedeljkovic', 'Tristan Jarry']
team_capitals=['Alexander Ovechkin', 'Dylan Strome', 'Matthew Phillips', 'T.J. Oshie', 'Yevgeni Kuznetsov', 'Connor McMicheal', 'Sonny Milano', 'Nicklas Backstrom', 'Tom Wilson', 'Anthony Mantha', 'Nick Dowd', 'Beck Malenstyn', 'Martin Fehervary', 'John Carlson', 'Trevor Van Riemsdyk', 'Nick Jensen', 'Alexander Alexeyev', 'Rasmus Sandin', 'Darcy Kuemper', 'Hunter Shepard']

In [ ]:
team_players={'COL':team_colorado, }